In [14]:
import random
import math
import csv
import copy

In [15]:
class Item:
    def __init__(self, itemid, width, height, depth, weight, profit):
        #assuming that along x axis is width, along y axis is height and along z axis is depth/length
        self.itemid = int(itemid)
        self.width = int(width)
        self.height = int(height)
        self.depth = int(depth)
        self.profit = int(profit)
        self.weight = int(weight)
        self.x = None
        self.y = None
        self.z = None
        
    def __repr__(self):
        return f"Item({self.itemid}, {self.width}, {self.height}, {self.depth}, ({self.x}, {self.y}, {self.z}))"


In [16]:
class Uld:
    #assuming that along x axis is width, along y axis is height and along z axis is depth/length
    def __init__(self, uid, width, height, depth, capacity):
        self.uid = int(uid)
        self.capacity = int(capacity)
        self.width = int(width)
        self.height = int(height)
        self.depth = int(depth)

In [17]:
def createSequenceTable(A,B,C):
    n = len(A)
    dictA = {}
    for i in range(n):
        dictA[A[i]] = i+1
    
    dictB = {}
    for i in range(n):
        dictB[B[i]] = i+1

    dictC = {}
    for i in range(n):
        dictC[C[i]] = i+1
    
    table = [dictA, dictB, dictC]
    return table

In [18]:
def canPlaceItemInUld(item, uld):
    if item.width>uld.width or item.height>uld.height or item.depth>uld.depth:
        return False
    if item.weight>uld.capacity:
        return False
    
    return True

In [ ]:
def itemIndex(tup, seq_index):
    item_id = tup[0]
    table = tup[1]
    dict_seq = table[seq_index]
    
    return dict_seq[item_id]

In [126]:
def transformTo3D(uid, i, placed_items_tot, table):
    px,py,pz=[],[],[]
    #checking each placed item with my current item
    placed_items = []
    for items in placed_items_tot:
        if(items[0]==uid):
            placed_items.append(items)
    for tup in placed_items:
        item_placed = tup[1]
        item = (i, table)
        j = (item_placed.itemid, table)
        
        if((itemIndex(item,0)<itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)>itemIndex(j,2))):
            px.append(tup)
        elif((itemIndex(item,0)>itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)<itemIndex(j,2)) ):
            py.append(tup)
        elif(((itemIndex(item,0)>itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)>itemIndex(j,2)))\
             or ((itemIndex(item,0)<itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)<itemIndex(j,2)) )):
            pz.append(tup)
    x,y,z=0,0,0


    for tup in px:
        j = tup[1]
        x = max(0, j.x + j.width)
    for tup in py:
        j = tup[1]
        y = max(0, j.y + j.height)
    for tup in pz:
        j = tup[1]
        z = max(0, j.z + j.depth)
    return x,y,z

In [201]:
def packUld(A,B,C, items, ulds, n, table):
    
    #n is the number of items
    placed_items = []
    score = 0
    total_weight = [0]*len(ulds)
    total_profit = 0
    for item_index_seq in range(n-1,-1,-1):
        curItem = items[B[item_index_seq]-1]
        placed = False
        for uld in ulds:
            x,y,z = transformTo3D(uld.uid, curItem.itemid, placed_items, table)
            if(x+curItem.width <= uld.width and\
               y+curItem.height <= uld.height and\
               z+curItem.depth <= uld.depth and\
               curItem.weight + total_weight[uld.uid - 1] <= uld.capacity
              ):
                curItem.x, curItem.y , curItem.z = x,y,z
                placed_items.append([uld.uid,curItem])
                score += curItem.profit
                total_profit += curItem.profit
                total_weight[uld.uid - 1] += curItem.weight
                placed = True
                break
        if not placed:
            #penalty for unplaced items
            total_profit -= 0.5*curItem.profit
                
    return score, total_profit, placed_items

In [202]:
def modifySequence(A,B,C):
    new_A = A[:]
    new_B = B[:]
    new_C = C[:]
    swap_ch = random.choice([0,1])
    if(swap_ch == 1):
        seq_choice = random.choice(['A','B','C'])

        if seq_choice == 'A':
            i,j = random.sample(range(len(A)), 2)
            new_A[i], new_A[j] = new_A[j], new_A[i]

        elif seq_choice == 'B':
            i,j = random.sample(range(len(A)), 2)
            new_B[i], new_B[j] = new_B[j], new_B[i]

        elif seq_choice == 'C':
            i,j = random.sample(range(len(A)), 2)
            new_C[i], new_C[j] = new_C[j], new_C[i]
    else:
        i,j = random.sample(range(len(A)), 2)
        new_A[i], new_A[j] = new_A[j], new_A[i]
        
        i,j = random.sample(range(len(A)), 2)
        new_B[i], new_B[j] = new_B[j], new_B[i]
        
        i,j = random.sample(range(len(A)), 2)
        new_C[i], new_C[j] = new_C[j], new_C[i]
        
    return new_A,new_B,new_C

In [216]:
def simulateAnnealing(items, ulds, max_iteration = 2000, temp = 1100, decrease_factor = 0.99):
    
    n = len(items)
    current_A = list(range(1,n+1))
    current_B = list(range(1,n+1))
    current_C = list(range(1,n+1))
    random.shuffle(current_A)
    random.shuffle(current_B)
    random.shuffle(current_C)
    table = createSequenceTable(current_A, current_B, current_C)

    items_copy = copy.deepcopy(items)
    ulds_copy = copy.deepcopy(ulds)
    score, current_profit, placed_items = packUld(current_A, current_B, current_C, items_copy, ulds_copy, n, table)
    #using slicing to avoid same reference
    best_A , best_B, best_C =  current_A[:], current_B[:], current_C[:]
    best_placing = copy.deepcopy(placed_items)
    best_profit = current_profit
    best_score = score
    T = temp

    for recur in range(max_iteration):
        new_A, new_B, new_C = modifySequence(current_A, current_B, current_C)
        table = createSequenceTable(new_A, new_B, new_C)
        new_score, new_profit, placed_items = packUld(new_A, new_B, new_C, items, ulds, n, table)
        delta = (new_profit-current_profit)
        
        if delta>0 or random.uniform(0,1) < math.exp(delta/T):
            current_A , current_B, current_C = new_A[:], new_B[:], new_C[:]
            current_profit = new_profit
            
            #if the solution is accepted with some probabilty then we do not update our best profits
            if new_profit>best_profit:
                best_A, best_B, best_C = new_A[:], new_B[:], new_C[:]
                best_score = new_score
                best_profit = new_profit
                best_placing = placed_items[:]
            
        T *= decrease_factor
        #print(table)
        #print(score)
        #print("---------------------------------------")
        if T < 1e-3:
            break
    print("Temperature :",T)
    return best_placing, best_score

In [217]:
#extracting data from file

uld_obj = open('uld.csv', 'r')
item_obj = open('packages.csv', 'r')
item_data = csv.reader(item_obj)
items=[]
ulds=[]
ind = 1
for row in item_data:
    if ind == 1:
        ind = 2
        continue
    row = [x.strip().replace("'", "").strip() for x in row]  # Remove quotes
    item = Item(*row) 
    items.append(item)
    
uld_item = csv.reader(uld_obj)
ind = 1
for row in uld_item:
    if(ind==1):
        ind = 2
        continue
    row = [x.strip().replace("'", "").strip() for x in row]  # Remove quotes
    container = Uld(*row) 
    ulds.append(container)
    
uld_obj.close()
item_obj.close()

In [218]:
'''#sample data
items = [
    item(itemid=1, height=10, width=5, depth=20, weight=15, profit=100),
    item(itemid=2, height=15, width=7, depth=12, weight=10, profit=80),
    item(itemid=3, height=8, width=6, depth=18, weight=12, profit=90),
    item(itemid=4, height=12, width=5, depth=15, weight=8, profit=70),
    item(itemid=5, height=7, width=4, depth=10, weight=5, profit=40),
]

# Create a list of ULDs
ulds = [
    uld(uid=1, width=20, height=30, depth=20, capacity=200),
    #uld(uid=2, width=10, height=20, depth=20, capacity=300),
]

'''

arrangement, score = simulateAnnealing(items, ulds)
noitem = len(arrangement)
print("No of packages sent :",noitem)
print("score: ", score)
print(arrangement)

Temperature : 0.0009911450190030686
No of packages sent : 32
score:  10773
[[1, Item(191, 37, 27, 50, (0, 0, 0))], [1, Item(73, 43, 30, 20, (0, 0, 50))], [2, Item(92, 38, 33, 33, (0, 0, 0))], [2, Item(104, 28, 36, 38, (0, 0, 33))], [1, Item(75, 21, 26, 23, (43, 0, 50))], [3, Item(37, 37, 41, 41, (0, 0, 0))], [3, Item(60, 30, 38, 40, (37, 0, 0))], [1, Item(45, 25, 44, 27, (64, 0, 0))], [2, Item(38, 48, 44, 30, (0, 36, 33))], [3, Item(148, 39, 40, 46, (0, 38, 41))], [2, Item(36, 30, 33, 38, (48, 0, 0))], [3, Item(113, 43, 28, 20, (0, 78, 0))], [2, Item(52, 44, 22, 34, (38, 0, 38))], [4, Item(86, 27, 31, 31, (0, 0, 0))], [4, Item(145, 47, 35, 31, (0, 0, 31))], [3, Item(67, 40, 27, 36, (0, 38, 20))], [5, Item(138, 20, 34, 22, (0, 0, 0))], [5, Item(53, 44, 30, 29, (0, 0, 22))], [4, Item(55, 35, 25, 43, (0, 35, 0))], [1, Item(159, 38, 32, 33, (37, 0, 27))], [3, Item(137, 22, 29, 50, (0, 0, 56))], [4, Item(13, 27, 42, 26, (0, 0, 43))], [4, Item(169, 21, 37, 25, (27, 0, 43))], [5, Item(27, 29,

In [161]:
#saving output to file

opfile = open("output11.csv", 'w', newline = '')
writer = csv.writer(opfile)
writer.writerow([score])
writer.writerow([noitem])
for row in arrangement:
    itemobj = row[1]
    uid = row[0]
    itemid = itemobj.itemid
    x0 = itemobj.x
    y0 = itemobj.y
    z0 = itemobj.z
    x1 = x0 + itemobj.width
    y1 = y0 + itemobj.height
    z1 = z0 + itemobj.depth
    writer.writerow([itemid, uid, x0, y0, z0, x1, y1, z1])
    
opfile.close()